# Import Packages

In [71]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, RFECV,RFE
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from matplotlib import pyplot
from sklearn.preprocessing import PowerTransformer, Normalizer , StandardScaler, MinMaxScaler

# Load Data 

In [112]:
data_train_org=pd.read_csv(r'D:\Transcend_(E)\_IramS\Covid Challenge\Imputed_data_train.csv', delimiter=',')
data_test_org = pd.read_csv (r'D:\Transcend_(E)\_IramS\Covid Challenge\only_images.csv', delimiter=',')

entries= Path('D:\Transcend_(E)\_IramS\Covid Challenge')
os.chdir(entries)

In [113]:
Imaging_train_org=pd.read_csv(r'D:\Transcend_(E)\_IramS\Covid Challenge\Densnet_Threshold_Image_features.csv', delimiter=',')
Imaging_test_org = pd.read_csv (r'D:\Transcend_(E)\_IramS\Covid Challenge\Densnet_Threshold_Image_features_test.csv', delimiter=',')

Img_data_train= Imaging_train_org[['type_1011','type_179','type_296','type_336','type_47','type_508','type_78','type_880','type_971','type_974']]  # 'dzm_zdnu_norm_3d_fbn_n32 ''log_stat_min' dzm_lde_3d_fbn_n32' ivh_v10 morph_geary_c log_ih_skew_fbn_n32   log_rlm_lrhge_3d_v_mrg_fbn_n32                                                         
Img_data_test = Imaging_test_org[['type_1011','type_179','type_296','type_336','type_47','type_508','type_78','type_880','type_971','type_974']]

Data processing through out the script will be performed on pandas dataframes. We want to keep dataframe format so we can extract dataframe headers i.e. feature names at filtering steps. This will help to build record of selected features. 

In [114]:
Img_data_test

,type_1011,type_179,type_296,type_336,type_47,type_508,type_78,type_880,type_971,type_974
0,0.160479,0.000092,0.970427,0.000686,0.000108,0.080041,0.001353,1.220457,1.128150,0.650273
1,0.000000,0.000078,1.075327,0.000452,0.000107,0.057388,0.001515,0.774492,0.185469,0.333532
2,0.003175,0.000107,0.969194,0.000660,0.000102,0.126317,0.001407,0.670949,1.041468,0.767904
3,0.000312,0.000066,1.173416,0.000734,0.000096,0.054399,0.001253,0.259640,0.934348,0.396822
4,0.000000,0.000061,1.468449,0.001088,0.000076,0.001160,0.001218,0.123152,0.345886,0.089603
...,...,...,...,...,...,...,...,...,...,...
84,0.098011,0.000032,1.261685,0.000825,0.000078,0.053169,0.001188,0.046369,0.421491,0.005307
85,0.023412,0.000102,0.976839,0.000557,0.000111,0.257167,0.001633,1.014529,1.288849,1.762612
86,0.043563,0.000067,0.874347,0.000523,0.000114,0.140542,0.001230,0.984644,1.214913,0.556053
87,0.004709,0.000079,1.554062,0.000836,0.000085,0.000000,0.000851,1.036133,0.066823,0.228431


In [115]:
#clinical_train = pd.read_excel(r'E:\_IramS\Dresden_Data\Feature_Extraction_MR\Feature_Extraction_MR_DD\TRG\MRI_Robust_0.80_Clustering_Max_AUC_TRG.xlsx')
#clinical_test = pd.read_excel(r'E:\_IramS\Dresden_Data\Feature_Extraction_MR\Feature_Extraction_MR_MTU\TRG\MTU_MRI_Robust_0.80_Clustering_Max_AUC_TRG.xlsx')


#clinical_train= clinical_train[['Grade','cT_binary']]
#clinical_test= clinical_test[['Grade','cT_binary']]

In [116]:
data_train= data_train_org[['LDH','Ox_percentage','SaO2','CRP','PaO2']]  # 'dzm_zdnu_norm_3d_fbn_n32 ''log_stat_min' dzm_lde_3d_fbn_n32' ivh_v10 morph_geary_c log_ih_skew_fbn_n32   log_rlm_lrhge_3d_v_mrg_fbn_n32                                                         
data_test = data_test_org[['LDH','Ox_percentage','SaO2','CRP','PaO2']]

In [117]:
data_test

,LDH,Ox_percentage,SaO2,CRP,PaO2
0,NaN,93.0,93.0,3.96,63.0
1,NaN,NaN,94.0,5.20,65.0
2,NaN,NaN,NaN,13.72,53.0
3,NaN,NaN,98.0,NaN,90.0
4,NaN,84.0,NaN,14.76,49.0
...,...,...,...,...,...
84,NaN,NaN,98.0,3.13,94.0
85,178.0,88.0,NaN,2.60,83.0
86,NaN,89.0,93.0,20.76,60.0
87,NaN,NaN,77.0,9.52,41.0


In [118]:
lables_train= pd.Series(np.where(data_train_org.Prognosis.values == 'MILD', 0, 1),data_train_org.index)

In [119]:
X_train= Img_data_train # Get train data in pd frame 
y_train= lables_train
X_test= Img_data_test # Get test data in pd frame 
y_test= data_test_org.iloc[:,-1] 
#X_test.shape, y_test.shape # check shape
y_test

0     0
1     0
2     0
3     0
4     0
     ..
84    0
85    0
86    0
87    0
88    0
Name: Prognosis, Length: 89, dtype: int64

In [120]:
def transform_fun(X_train,X_test):
    
    yeo_j_trans= PowerTransformer()
    yeo_j_trans.fit(X_train) # fit to train data 
    X_train_trans= yeo_j_trans.transform(X_train) # transform train data 
    X_train_trans_df = pd.DataFrame(X_train_trans, columns = X_train.columns, index=X_train.index) # convert to dataframe 
    X_test_trans= yeo_j_trans.transform(X_test) # transform test
    X_test_trans_df = pd.DataFrame(X_test_trans, columns = X_test.columns, index=X_test.index) #convert to dataframe

    return X_train_trans_df, X_test_trans_df

# Scaling

In [121]:
def scaling_data(X_train, X_test):
    
    #scaler= MinMaxScaler()
    scaler = StandardScaler()
    scaler.fit(X_train)# fit to train 
    X_train_scaled= scaler.transform(X_train)# transform train
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns = X_train.columns, index=X_train.index) #convert train to dataframe 
    X_test_scaled= scaler.transform(X_test) #transform test
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns = X_test.columns, index=X_test.index) #convert test to dataframe 
    
    return X_train_scaled_df, X_test_scaled_df

In [122]:
def recurssive_selction(X_train, y_train, X_test): 
    sig_features_rfe=[]
    rfe=RFE(estimator=LogisticRegression(),n_features_to_select='all')
    rfe.fit(X_train, y_train)
    mask = rfe.get_support() #get a mask of selected features
    features_rfe = X_train.columns[mask] # get feature names 
    sig_features_rfe.append(features_rfe) # store in list
    
    X_train_rfe= rfe.transform(X_train) #transform train features
    X_test_rfe= rfe.transform(X_test)   #transform test features
        
    X_train_rfe_df = pd.DataFrame(X_train_rfe, columns = features_rfe, index=X_train.index) #convert train to dataframe 
    X_test_rfe_df= pd.DataFrame(X_test_rfe, columns=features_rfe, index=X_test.index)     #convert test to dataframe
    
    return X_train_rfe_df, X_test_rfe_df, sig_features_rfe

# Fitting Logistic Regression model on Selected features 

# Fit model and Compute Evaluation metrices 

In [123]:
def fit_compute_results(X_train, y_train, X_test, y_test):
        
        model = LogisticRegression(solver='lbfgs',penalty='none',class_weight='balanced')
        model.fit(X_train, y_train)
    

        # Getting Prediction Probabilties for Each Repetition on Training and Test Set of Selected Features
        proba_train= model.predict_proba(X_train)
        proba_test= model.predict_proba(X_test)
    
        # Getting Train and Test Probabilites for positive class to calculate train and test auc 
        yhat_train= proba_train[:,1]
        yhat_test= proba_test[:,1]
    
        
        return yhat_train,yhat_test

In [124]:
X_train_trans_df, X_test_trans_df = transform_fun(X_train, X_test)


# Scaling features
X_train_scaled_df, X_test_scaled_df = scaling_data(X_train_trans_df, X_test_trans_df)
            
yhat_train, yhat_test = fit_compute_results(X_train_scaled_df,y_train,X_test_scaled_df,y_test)


In [125]:
yhat_test

array([0.36092819, 0.41184333, 0.3114557 , 0.485033  , 0.81700159,
       0.73909668, 0.26528362, 0.50445203, 0.54847996, 0.62614438,
       0.37863451, 0.69932855, 0.5173515 , 0.34961366, 0.09243455,
       0.78968617, 0.63991934, 0.51150724, 0.2804359 , 0.65652466,
       0.22641021, 0.47777761, 0.81453142, 0.64873995, 0.67675492,
       0.74044138, 0.58520156, 0.40579557, 0.59680087, 0.70744516,
       0.24369455, 0.63274006, 0.173556  , 0.54653448, 0.68128276,
       0.35858988, 0.69032505, 0.68193984, 0.38653291, 0.50459528,
       0.29067127, 0.43303873, 0.33003615, 0.47401217, 0.27949685,
       0.37034333, 0.49535992, 0.80877789, 0.46121585, 0.62754003,
       0.29850152, 0.25698744, 0.72268223, 0.37527517, 0.55758279,
       0.36686165, 0.71395557, 0.45469441, 0.41378815, 0.22947836,
       0.38602089, 0.49786088, 0.2920884 , 0.27339151, 0.25824277,
       0.38878014, 0.29745108, 0.43018281, 0.71350052, 0.46690847,
       0.77398982, 0.61881305, 0.13448757, 0.41559955, 0.46246

In [126]:
proba = pd.DataFrame(yhat_test)
proba

,0
0,0.360928
1,0.411843
2,0.311456
3,0.485033
4,0.817002
...,...
84,0.681057
85,0.184766
86,0.211603
87,0.722496


In [127]:
full = pd.concat([data_test_org[['PatientID']], proba], axis=1)

In [128]:
full

,PatientID,0
0,P_102,0.360928
1,P_16,0.411843
2,P_118,0.311456
3,P_114,0.485033
4,P_2,0.817002
...,...,...
84,P_67,0.681057
85,P_92,0.184766
86,P_86,0.211603
87,P_9,0.722496


In [129]:
full.to_csv('results_of_imaging_only.csv', header=True)